In [ ]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, _ = nml.load_synthetic_multiclass_classification_dataset()

display(reference_df.head(3))

,id,acq_channel,app_behavioral_score,requested_credit_limit,app_channel,credit_bureau_score,stated_income,is_customer,timestamp,y_pred_proba_prepaid_card,y_pred_proba_highstreet_card,y_pred_proba_upmarket_card,y_pred,y_true
0,0,Partner3,1.808232,350,web,309,15000,True,2020-05-02 02:01:30,0.97,0.03,0.00,prepaid_card,prepaid_card
1,1,Partner2,4.382568,500,mobile,418,23000,True,2020-05-02 02:03:33,0.87,0.13,0.00,prepaid_card,prepaid_card
2,2,Partner2,-0.787575,400,web,507,24000,False,2020-05-02 02:04:49,0.47,0.35,0.18,prepaid_card,upmarket_card


In [ ]:
print(reference_df.head(3).to_markdown(tablefmt="grid"))

+----+------+---------------+------------------------+--------------------------+---------------+-----------------------+-----------------+---------------+---------------------+-----------------------------+--------------------------------+------------------------------+--------------+---------------+
|    |   id | acq_channel   |   app_behavioral_score |   requested_credit_limit | app_channel   |   credit_bureau_score |   stated_income | is_customer   | timestamp           |   y_pred_proba_prepaid_card |   y_pred_proba_highstreet_card |   y_pred_proba_upmarket_card | y_pred       | y_true        |
+====+======+===============+========================+==========================+===============+=======================+=================+===============+=====================+=============================+================================+==============================+==============+===============+
|  0 |    0 | Partner3      |               1.80823  |                      350 | web      

In [ ]:
# matrix can be provided as a list of lists or a numpy array
business_value_matrix = [
    [1, 0, -1],
    [0, 1, 0],
    [-1, 0, 1]
]
estimator = nml.CBPE(
    y_pred_proba={
        'prepaid_card': 'y_pred_proba_prepaid_card',
        'highstreet_card': 'y_pred_proba_highstreet_card',
        'upmarket_card': 'y_pred_proba_upmarket_card'},
    y_pred='y_pred',
    y_true='y_true',
    timestamp_column_name='timestamp',
    problem_type='classification_multiclass',
    metrics=['business_value'],
    business_value_matrix=business_value_matrix,
    normalize_business_value="per_prediction",
    chunk_size=6000,
)

In [ ]:
estimator.fit(reference_df)

In [ ]:
results = estimator.estimate(analysis_df)
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:5999]           0           0      5999 2020-09-01 03:10:01   
1   [6000:11999]           1        6000     11999 2020-09-13 16:15:32   
2  [12000:17999]           2       12000     17999 2020-09-25 19:50:04   
3  [18000:23999]           3       18000     23999 2020-10-08 02:57:34   
4  [24000:29999]           4       24000     29999 2020-10-20 15:49:06   
5  [30000:35999]           5       30000     35999 2020-11-01 22:04:59   
6  [36000:41999]           6       36000     41999 2020-11-14 03:55:49   
7  [42000:47999]           7       42000     47999 2020-11-26 09:19:22   
8  [48000:53999]           8       48000     53999 2020-12-08 14:34:25   
9  [54000:59999]           9       54000     59999 2020-12-20 18:31:09   

                                business_value                          \
             end_date    period          value sampling_error realized   
0 2020-09-13 16:15:10  analysis       2.008617       0.008047      NaN   
1 2020-09-25 19:48:42  analysis       2.016709       0.008047      NaN   
2 2020-10-08 02:53:47  analysis       2.025152       0.008047      NaN   
3 2020-10-20 15:48:19  analysis       2.018928       0.008047      NaN   
4 2020-11-01 22:04:40  analysis       2.006521       0.008047      NaN   
5 2020-11-14 03:55:33  analysis       1.564443       0.008047      NaN   
6 2020-11-26 09:19:06  analysis       1.568460       0.008047      NaN   
7 2020-12-08 14:33:56  analysis       1.562041       0.008047      NaN   
8 2020-12-20 18:30:30  analysis       1.566866       0.008047      NaN   
9 2021-01-01 22:57:55  analysis       1.574250       0.008047      NaN   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  2.032760                  1.984475        2.050316   
1                  2.040851                  1.992566        2.050316   
2                  2.049294                  2.001010        2.050316   
3                  2.043070                  1.994786        2.050316   
4                  2.030664                  1.982379        2.050316   
5                  1.588585                  1.540300        2.050316   
6                  1.592603                  1.544318        2.050316   
7                  1.586183                  1.537898        2.050316   
8                  1.591009                  1.542724        2.050316   
9                  1.598392                  1.550107        2.050316   

                          
  lower_threshold  alert  
0        1.963201  False  
1        1.963201  False  
2        1.963201  False  
3        1.963201  False  
4        1.963201  False  
5        1.963201   True  
6        1.963201   True  
7        1.963201   True  
8        1.963201   True  
9        1.963201   True

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+---------------------+------------+--------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                     |            | | business_value   |                    |              |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date          | | period   | | value            | | sampling_error   | | realized   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============

In [ ]:
metric_fig = results.plot()
metric_fig.show()

In [ ]:
metric_fig = results.plot()
metric_fig.write_image(file=f"../_static/tutorials/performance_estimation/multiclass/business_value.svg")